LangChain ReAct Agent Demo

In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [12]:
# First we initialize the model we want to use.
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)


# For this tutorial we will use custom tool that returns pre-defined values for weather in two cities (NYC & SF)

from typing import Literal

from langchain_core.tools import tool

In [13]:
import os
import shutil
from typing import List, Dict, Optional

@tool
def list_files(directory: str) -> List[Dict[str, Optional[str]]]:
    """
    Lists all files in the specified directory, including their name, type, and metadata.
    """
    files = []
    for entry in os.scandir(directory):
        file_info = {
            "name": entry.name,
            "type": "directory" if entry.is_dir() else "file",
            "size": os.path.getsize(entry) if not entry.is_dir() else None,
            "created": os.path.getctime(entry),
            "modified": os.path.getmtime(entry),
        }
        files.append(file_info)
    return files


@tool
def create_folder(directory: str, folder_name: str) -> str:
    """
    Creates a new folder in the specified directory.
    """
    new_folder_path = os.path.join(directory, folder_name)
    os.makedirs(new_folder_path, exist_ok=True)
    return new_folder_path


@tool
def rename_item(old_path: str, new_name: str) -> str:
    """
    Renames a file or folder to the specified new name.
    """
    new_path = os.path.join(os.path.dirname(old_path), new_name)
    os.rename(old_path, new_path)
    return new_path


@tool
def change_directory(new_directory: str) -> str:
    """
    Changes the working directory to the specified path.
    """
    os.chdir(new_directory)
    return os.getcwd()


@tool
def move_item(src_path: str, dest_directory: str) -> str:
    """
    Moves a file or folder to the specified destination directory.
    """
    dest_path = os.path.join(dest_directory, os.path.basename(src_path))
    shutil.move(src_path, dest_path)
    return dest_path


In [14]:
tools = [list_files, create_folder, rename_item, change_directory, move_item]

# Define the graph

from langgraph.prebuilt import create_react_agent

graph = create_react_agent(model, tools=tools)

In [15]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [16]:
inputs = {"messages": [("user", "There is a folder called 'downloads' in this directory. Use your tools to sort the files into folders based on categories you determine. If you don't know what to do with a file, don't delete it - just move it to a 'miscellaneous' folder. Keep all your sub-folders within downloads.")]}
print_stream(graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

There is a folder called 'downloads' in this directory. Use your tools to sort the files into folders based on categories you determine. If you don't know what to do with a file, don't delete it - just move it to a 'miscellaneous' folder. Keep all your sub-folders within downloads.
================================== Ai Message ==================================
Tool Calls:
  list_files (call_a5lCo5FL6jv7EvE64sa4DSgQ)
 Call ID: call_a5lCo5FL6jv7EvE64sa4DSgQ
  Args:
    directory: downloads
================================= Tool Message =================================
Name: list_files

[{"name": "egl47_resume.pdf", "type": "file", "size": 104028, "created": 1730693269.0491664, "modified": 1727670988.9579325}, {"name": "error_3350.txt", "type": "file", "size": 79930, "created": 1730693269.0667443, "modified": 1725418037.151453}, {"name": "Everett_Lee_Resume.pdf", "type": "file", "size": 64316, "created": 1